# 04 Facturas
Este algoritmo está orientado a tomar las facturas del reporte correspondiente, pero validando que las referencias, importes y fechas sean consistentes antes de ligarlo, e indicarle al usuario por qué se ligó o por qué no se ligó. 

In [ ]:
# Importa librerías y chromedriver
import sys
import os
import glob
import yaml
import pandas as pd
from datetime import datetime
import shutil
import time

# Define the root and script directory
folder_root = os.getcwd()  # Get current directory (where Orquestación.ipynb is)
script_folder = os.path.join(folder_root, "Scripts")  # Path to 'Scripts'
# Ensure the script folder is added to sys.path
if script_folder not in sys.path:
    sys.path.append(script_folder)
Libreria_facturas = os.path.join(script_folder, "Libreria_facturas")
if Libreria_facturas not in sys.path:
    sys.path.append(Libreria_facturas)
Libreria_comunes = os.path.join(script_folder, "Libreria_comunes")
if Libreria_comunes not in sys.path:
    sys.path.append(Libreria_comunes)

In [ ]:
# Cargar información general del YAML
folder_implementacion = os.path.join(folder_root, "Implementación")
folder_facturas = os.path.join(folder_implementacion, "Facturas")
output_yaml = os.path.join(folder_root, "Implementación", "Facturas", "facturas.yaml")
yaml_exists = os.path.exists(output_yaml)

if yaml_exists:
    # Abrir y cargar el contenido YAML en un diccionario
    with open(output_yaml, 'r', encoding='utf-8') as f:
        paths_y_sheets = yaml.safe_load(f)
    #print(f"Archivo cargado correctamente: {os.path.basename(output_yaml)}")
    #print("Contenido:", data_access)
    #sys.exit()

else: 
    secciones = ["PAQ_IMSS_2023", "PAQ_IMSS_2024", "PAQ_IMSS_2025", "PAQ_INSABI_2023", "PAQ_INSABI_2024", "PAQ_INSABI_2025", "PAQ_CCINSHAE_2023", "PAQ_CCINSHAE_2024", "PAQ_CCINSHAE_2025"]
    variables    = ["Dropbox", "Excel_sheet"]
    
    lines = []
    for seccion in secciones:
        for variable in variables:
            # clave = valor vacío
            lines.append(f"{seccion}_{variable}: ")
        lines.append("")  # línea en blanco entre bloques
    
    # Escribe el archivo YAML (aunque use "=" tal como en tu ejemplo)
    with open(output_yaml, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))
    
    print(f"Archivo generado correctamente en: {output_yaml}")


El yaml contiene la ruta del archivo y el nombre de la hoja de excel que contiene el reporte, por ejemplo: 
```
PAQ_IMSS_2023_Dropbox: 'C:\Users\arman\Dropbox\FACT 2023\Generacion facturas IMSS VFinal.xlsx'
PAQ_IMSS_2023_Excel_sheet: 'Reporte Paq'
```

In [ ]:
# Generar banco de XML's
from xml_extraction import smart_xml_extraction
invoice_paths = [
        r'C:\Users\arman\Dropbox\FACT 2023',
        r'C:\Users\arman\Dropbox\FACT 2024',
        r'C:\Users\arman\Dropbox\FACT 2025']
xlsx_database = os.path.join(folder_root, "Implementación", "Facturas", 'xmls_extraidos.xlsx')
smart_xml_extraction(invoice_paths, xlsx_database)

## IMSS: Reporte de facturas

In [ ]:
# Importar la función y ligar órdenes y facturas. 
from folders_files_open import create_directory_if_not_exists
from paqs_validation import validacion_de_paqs
xlsx_database = os.path.join(folder_root, "Implementación", "Facturas", 'xmls_extraidos.xlsx')

IMSS_PAQS = {
    paths_y_sheets.get("PAQ_IMSS_2023_Dropbox"): paths_y_sheets.get("PAQ_IMSS_2023_Excel_sheet"),
    paths_y_sheets.get("PAQ_IMSS_2024_Dropbox"): paths_y_sheets.get("PAQ_IMSS_2024_Excel_sheet"),
    paths_y_sheets.get("PAQ_IMSS_2025_Dropbox"): paths_y_sheets.get("PAQ_IMSS_2025_Excel_sheet"),
}
facturas_IMSS = os.path.join(folder_facturas, 'IMSS')
create_directory_if_not_exists(facturas_IMSS)
IMSS_columas = {
    'IMSS_2023': ['Folio', 'Referencia','Alta', 'Total','UUID'], 
    'IMSS_2024': ['Folio', 'Referencia','Alta', 'Total', 'UUID'],
    'IMSS_2025': ['Folio', 'Referencia','Alta', 'Total', 'UUID']
}
altas_path = os.path.join(folder_implementacion, "SAI", "Ordenes_altas.xlsx") 
altas_sheet = 'df_altas'
df_altas = pd.read_excel(altas_path, sheet_name=altas_sheet)
info_types = 'IMSS'
#print(altas_path)
#print(IMSS_PAQS, '\n', IMSS_columas, '\n', facturas_IMSS, df_altas)

validacion_de_paqs(IMSS_PAQS, IMSS_columas, facturas_IMSS, altas_path, altas_sheet, info_types, xlsx_database)